In [1]:
"""
Zone 5, step 7, target defintion 
Proportion of customers with at least one positive target
"""


'\nZone 5, step 7, target defintion \nProportion of customers with at least one positive target\n'

In [2]:
from helper_functions import start_spark_session, get_imputed_df, suffix_and_join_dfs
from col_stats import *
from pyspark.sql.functions import col, round,  countDistinct

In [3]:
spark = start_spark_session()

In [4]:
def main(imputed_df, spark):
    
    per_ctu_count_partyids_with_positive_interactions = imputed_df.where("te_2month = 1").\
        groupBy('CTU').agg(countDistinct('party_id'))

    per_ctu_count_partyids_with_all_interactions = imputed_df.\
        groupBy('CTU').agg(countDistinct('party_id'))

    
    joined_df = suffix_and_join_dfs( per_ctu_count_partyids_with_positive_interactions,
                                   per_ctu_count_partyids_with_all_interactions, 'CTU')

    proportion_of_positives = calculate_proportion_of_partyids_of_positives_per_ctu (joined_df )
    proportion_of_positives.show()
    ctus_all = list(imputed_df.select('CTU').distinct().collect())
    proportion_of_positives_full_table = add_zero_proportions_to_empty_ctus (proportion_of_positives , ctus_all, spark)
    return proportion_of_positives_full_table
        

In [5]:
def calculate_proportion_of_partyids_of_positives_per_ctu( joined_df ):
        proportion_of_positives = joined_df.withColumn("proportion_with_positive_target", 
                                     round(col('count(party_id)')/ 
                                           col('count(party_id)_2'),2))\
                                    .select('CTU',"proportion_with_positive_target")
        
        return proportion_of_positives
    
def add_zero_proportions_to_empty_ctus (proportion_of_positives , ctus_all, spark) :
   
    for ctu in sorted(ctus_all):
        ctu_query = "ctu == {0}".format(ctu[0])
        ctu_value  = proportion_of_positives.where(ctu_query).select('CTU').collect()
        if len(ctu_value) ==0:
            new_df = spark.createDataFrame([[ctu[0],0]] , ['CTU','proportion_with_positive_target'])
            proportion_of_positives = proportion_of_positives.union(new_df)
    return proportion_of_positives



def test_main_function(spark): 
    """
    """

    imputed_df = spark.createDataFrame([(1,1,0),(1,1,1),(1,1,1),
                                        (2,1,1),(2,2,0),
                                            (3,1,1),(3,2,0),
                                         (3,3,1),(3,3,0),
                                        (4,3,0),(4,3,0)
                                                  ], ['party_id', 'ctu', 'te_2month'])
    proportion_of_positives_full_table = main(imputed_df, spark)
    proportion_of_positives_full_table.show()
    return proportion_of_positives

In [6]:

imputed_df = get_imputed_df()
proportion_of_positives  = main(imputed_df, spark)
proportion_of_positives.show()

+----+-------------------------------+
| CTU|proportion_with_positive_target|
+----+-------------------------------+
| 1.0|                           0.48|
|15.0|                           0.47|
| 0.0|                           0.48|
| 9.0|                           0.49|
|12.0|                           0.48|
|10.0|                           0.46|
|14.0|                           0.49|
| 5.0|                           0.49|
| 6.0|                           0.48|
| 4.0|                           0.49|
| 7.0|                            0.5|
|11.0|                           0.46|
| 2.0|                           0.48|
| 8.0|                           0.49|
| 3.0|                            0.5|
|13.0|                            0.5|
+----+-------------------------------+

+----+-------------------------------+
| CTU|proportion_with_positive_target|
+----+-------------------------------+
| 1.0|                           0.48|
|15.0|                           0.47|
| 0.0|                  

In [7]:
test_main_function(spark)

+---+-------------------------------+
|CTU|proportion_with_positive_target|
+---+-------------------------------+
|  1|                            1.0|
|  3|                            0.5|
+---+-------------------------------+

+---+-------------------------------+
|CTU|proportion_with_positive_target|
+---+-------------------------------+
|  1|                            1.0|
|  3|                            0.5|
|  2|                            0.0|
+---+-------------------------------+



DataFrame[CTU: string, proportion_with_positive_target: double]